In [3]:
import math
import numpy as np
import matplotlib.pyplot as plt
#from scipy.special import softmax
#import note_seq
#from note_seq.protobuf import music_pb2
import pandas as pd

%matplotlib inline
if 'google.colab' in str(get_ipython()):
  %run drive/My\ Drive/agent-based-tonal-model/utility-functions.ipynb
else:
  %run utility-functions.ipynb

ERROR:root:File `'../drive/My Drive/agent-based-tonal-model/utility-functions.ipynb.py'` not found.


In [ ]:
###################################################
# TonalAgent.
# Produces single note [0-11] at each time step.
###################################################

def rotate_prob_distribution(p, ref_pitch):
    p_rotated = np.concatenate((p[-ref_pitch:],p[:-ref_pitch]))
    return p_rotated

def softmax2(z,base):
    z_exp = [base**i for i in z]
    z_sum = np.sum(z_exp)
    return z_exp/z_sum

def softmax(z,gamma):
    z_exp = [math.exp(i*gamma) for i in z]
    z_sum = np.sum(z_exp)
    return z_exp/z_sum

class TonalAgent:
    def __init__(self,tonal_map=diss_dict_chew_2_11,memory_=1,tau_=None,listen_weight_=1,gamma_=1):
        self.prev_notes = []
        self.other_notes = []
        self.memory = memory_ # constant representing how far back to look
        self.tonal_map = tonal_map
        prob = [-i for i in self.tonal_map.values()] # not used in new implementations
        self.listen_weight=listen_weight_
        self.gamma = gamma_
        self.prob = np.concatenate((prob,np.flip(prob[1:-1]))) # not used in new implementations
        self.tau = tau_
        return
    
    def seed(self,num_notes=50):
        # seeds in C
        self.prev_notes = [0]*num_notes
        return self.prev_notes
        
    def generate_next_note_rand(self):
        # returns a number 0-11 representing a note from chromatic scale
        note = np.random.randint(0,12)
        self.prev_notes.append(note)
        return note
    
    def generate_next_note(self):
        if len(self.prev_notes)==0:
            return self.generate_next_note_rand()        
        prob_self = [0]*12
        for m in range(self.memory):
            if ((m+1) > len(self.prev_notes)): continue
            prob_self += softmax(rotate_prob_distribution(self.prob, self.prev_notes[-(m+1)]),self.gamma)
        prob_self = prob_self/np.sum(prob_self)        
        note = np.random.choice(12,1,p=prob_self)[0]
        self.prev_notes.append(note)
        return note
    
    def generate_next_note_listen(self):
        # uses Approach2 from explore-note-generation
        if len(self.prev_notes)==0:
            return self.generate_next_note_rand()
        # compute probabilities from self
        prob_self = [0]*12
        for m in range(self.memory):
            if ((m+1) > len(self.prev_notes)): continue
            prob_self += softmax(rotate_prob_distribution(self.prob, self.prev_notes[-(m+1)]),self.gamma)
        
        prob_other = [0]*12
        for m in range(self.memory):
            if ((m+1) > len(self.other_notes)): continue
            prob_other += softmax(rotate_prob_distribution(self.prob, self.other_notes[-(m+1)]),self.gamma)
        prob_combined = (prob_self+prob_other)/np.sum(prob_self+prob_other)
        note = np.random.choice(12,1,p=prob_combined)[0]
        self.prev_notes.append(note)
        return note
    
    def generate_next_note_listen_no_self(self):
        # uses Approach2 from explore-note-generation
        if len(self.other_notes)==0:
            return self.generate_next_note_rand()

        prob_other = [0]*12
        for m in range(self.memory):
            if ((m+1) > len(self.other_notes)): continue
            prob_other += softmax(rotate_prob_distribution(self.prob, self.other_notes[-(m+1)]),self.gamma)
        prob_other = prob_other/np.sum(prob_other)
        note = np.random.choice(12,1,p=prob_other)[0]
        self.prev_notes.append(note)
        return note
    
    def generate_next_note_max_cons_no_self(self):
        if len(self.other_notes)==0:
            return self.generate_next_note_rand()
        if self.memory > len(self.other_notes):
            note_set = self.other_notes[-len(self.other_notes):]
        else:
            note_set = self.other_notes[-self.memory:]

        cons_scores = [None]*12
        for i in range(len(cons_scores)):
            cons_scores[i] = individual_consonance_set(note_set+[i])
        if (self.gamma=="deterministic"):
            note = cons_scores.index(max(cons_scores))
            self.prev_notes.append(note)
            return note
        prob = softmax(cons_scores, self.gamma)
        note = np.random.choice(12,1,p=prob)[0]
        self.prev_notes.append(note)
        return note
    
    def generate_next_note_temporal_decay(self):
        # get weighted pc_list
        pc_hist = [0]*12
        for i in range(len(self.other_notes)):
            note = self.other_notes[-(i+1)]
            weight = np.exp(-i/self.tau)
            pc_hist[note] += weight

        # use pc_list to compute hypothetical cons for each of 12 pitches
        cons_vals = []
        cur_weight = 1
        for i in range(12):
            pc_i = pc_hist.copy()
            pc_i[i] += cur_weight
            cons_vals.append(get_consonance(pc_i,self.tonal_map))

        if (self.gamma=="deterministic"):
            note = cons_scores.index(max(cons_scores))
            self.prev_notes.append(note)
            return note
        
        # pass these hypothetical cons values through softmax
        prob = softmax(cons_vals, self.gamma)
        next_note = np.random.choice(12,1,p=prob)[0]
        self.prev_notes.append(next_note)
        return next_note
    
    def listen(self,note):
        self.other_notes.append(note)